## Algorand Covid-19 Project
## Demo: Basic Data Processing
Source documentation: https://github.com/algorandfoundation/IReport-Covid/blob/master/js/retrieveData.js

In [3]:
import pandas as pd
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
date = str(datetime.today().strftime('%Y-%m-%d'))

In [4]:
data_df = pd.read_csv('data/covidData'+date+'.csv')
print(data_df.shape)
data_df.head()

(1282, 42)


,_t,_v,tx_id,consent,age_group,country_code,region_code,gender,3_dig_zip,doctors_office,walk_in_clinic,virtual_care,hospital_or_ER,other,hospitalized,when_discharged,still_in_hospital,when_admitted,received_care,symptom_quarantine,voluntary_quarantine,personally_required_quarantine,general_quarantine,when_quarantine_ended,still_in_quarantine,when_quarantine_started,left_quarantine_temporarily,was_quarantined,fever,cough,difficulty_breathing,fatigue,sore_throat,when_symptoms_ended,still_symptomatic,when_symptoms_started,is_symptomatic,test_date,test_location,test_results,tried_to_get_tested,tested
0,report,1,UGY5YWRRYLLCDC5SD2BIAUEBXKZROG4N56VT6QRZBZD566...,1.0,55.0,US,NY,f,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,1.0,NaN,NaN,NaN,1.0,2020-03-10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
1,report,1,5USYC4OOGQYGSKRS3LLTRCSLYIX5CY6Y2HIDRY6S3MISED...,1.0,40.0,AU,ACT,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
2,report,1,MSJEL2ZTRSNKJLA6G5H5ITPLY6ZFYBFVG47G3765XM7OQJ...,1.0,65.0,US,NaN,m,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,True,NaN,NaN,NaN,2020-03-20,NaN,2020-03-13,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,2020-03-20,3.0,-1.0,NaN,1.0
3,report,1,FJD33KOQXQ5E2SA5VWB5MAIHXZTKVZUM5NEK7FJN56UKGS...,1.0,40.0,US,NY,f,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
4,report,1,QTLPDQHRDHMD5NGGIZFSRZ4MQACT3MK76JZ6JMDXDNLIRZ...,1.0,20.0,US,MD,m,207.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0


In [5]:
def clean_df(df):
    if (len(list(set(df.consent))) == 1) & (list(set(df.consent))[0] == True):
        print("consent given for all data")
    else:
        print("error")
        
    df = df.drop(['_t','_v','consent'],axis = 1)
    
    bool_cols = ['doctors_office','walk_in_clinic','virtual_care','hospital_or_ER','other','still_in_hospital',
                'symptom_quarantine','voluntary_quarantine','personally_required_quarantine','general_quarantine','still_in_quarantine',
                "fever","cough","difficulty_breathing","fatigue","sore_throat","still_symptomatic"]
                 
    for col in bool_cols:
        df[col]= df[col].replace(True, 1)
        df[col]= df[col].replace(False, 0)
    
    return df
    

In [6]:
# use anon 3 digit zipcode to try and impute state abbrevs
def long_lat(df):
    lat_long_df = pd.read_csv(r'C:\Users\rayzc\Downloads\us-zip-code-latitude-and-longitude.csv',sep=';')
    states = []
    match = 0
    for state,area in zip(df.region_code,df['3_dig_zip']):
        region = state
        if (str(region)=='nan') & (str(area)!='nan'):
            for s,z in zip(lat_long_df.State,lat_long_df.Zip):
                if (str(int(area)) in str(int(z))):
                    region = s
                    #print(str(int(area)),'matched with',region)
                    match+=1
                    break
        states.append(region)

    df.insert(3, "state_code", states)
    print("{} US 3-dig-zipcodes w/ missing region codes matched with a state".format(match))
    
    all_nan_inds = df.index[df.iloc[:,6:].isna().all(axis=1)]
    print("Removed {} rows with all nans in feature columns".format(len(all_nan_inds)))
    df = cleaner_data_df.drop(all_nan_inds,axis=0)
    
    
    return df

In [7]:
cleaner_data_df = clean_df(data_df)
cleaner_data_df = long_lat(cleaner_data_df)
print(cleaner_data_df.shape)
cleaner_data_df.head(10)

consent given for all data
140 US 3-dig-zipcodes w/ missing region codes matched with a state
Removed 10 rows with all nans in feature columns
(1272, 40)


,tx_id,age_group,country_code,state_code,region_code,gender,3_dig_zip,doctors_office,walk_in_clinic,virtual_care,hospital_or_ER,other,hospitalized,when_discharged,still_in_hospital,when_admitted,received_care,symptom_quarantine,voluntary_quarantine,personally_required_quarantine,general_quarantine,when_quarantine_ended,still_in_quarantine,when_quarantine_started,left_quarantine_temporarily,was_quarantined,fever,cough,difficulty_breathing,fatigue,sore_throat,when_symptoms_ended,still_symptomatic,when_symptoms_started,is_symptomatic,test_date,test_location,test_results,tried_to_get_tested,tested
0,UGY5YWRRYLLCDC5SD2BIAUEBXKZROG4N56VT6QRZBZD566...,55.0,US,NY,NY,f,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,1.0,NaN,NaN,NaN,1.0,2020-03-10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
1,5USYC4OOGQYGSKRS3LLTRCSLYIX5CY6Y2HIDRY6S3MISED...,40.0,AU,ACT,ACT,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
2,MSJEL2ZTRSNKJLA6G5H5ITPLY6ZFYBFVG47G3765XM7OQJ...,65.0,US,CO,NaN,m,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,2020-03-20,NaN,2020-03-13,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,2020-03-20,3.0,-1.0,NaN,1.0
3,FJD33KOQXQ5E2SA5VWB5MAIHXZTKVZUM5NEK7FJN56UKGS...,40.0,US,NY,NY,f,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
4,QTLPDQHRDHMD5NGGIZFSRZ4MQACT3MK76JZ6JMDXDNLIRZ...,20.0,US,MD,MD,m,207.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
5,T2BOPZENH2YNWHM3D32BDM7HUK347ZA6HNIAJPQV7FGIXR...,20.0,US,NY,NY,f,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,True,NaN,NaN,NaN,True,2020-03-09,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
6,5C2XEUOMBL2YVOH43OR7TOKJCBYKB5I6QAE5V5M4QKZNLF...,10.0,US,MD,MD,f,211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,True,NaN,1.0,NaN,True,2020-03-15,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
7,QJYU3JBCB7D2VGN5O2D5MYMSOKS7ZXPRFBSQVYT3PJJP4W...,20.0,US,MD,MD,f,212.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
8,MH43U3EGZ6BYTZOMWC3I2R4HJWSMFABR2FTBXND33YJCEY...,20.0,US,NY,NY,f,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,True,NaN,NaN,NaN,True,2020-03-14,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,-1.0,-1.0
9,CUCMP6J3H7BCNGEU5B2IGOGMNTRHWUZXR7YK2SGL7FHWAH...,20.0,US,WI,NaN,f,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,True,NaN,NaN,NaN,True,2020-03-19,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0


In [8]:
cleaner_data_df.to_csv('data/cleaner_covidData'+date+'.csv',index=False)